# CSE283 Final

## Load data

In [3]:
%run -i 'scripts/01_load.py'

No metadata file specified.


In [4]:
whos

Variable    Type        Data/Info
---------------------------------
data        dict        n=3
dataset     str         val
datasets    dict        n=3
files       tuple       n=3
load_data   function    <function load_data at 0x1667c9ee0>
os          module      <module 'os' from '/Users<...>203/lib/python3.9/os.py'>
pd          module      <module 'pandas' from '/U<...>ages/pandas/__init__.py'>


In [8]:
data['val']

{'counts':                  BCSC_S5_01  BCSC_S5_02  BCSC_S5_03  BCSC_S5_04  BCSC_S5_05  \
 ENSG00000223972           0           0           0           0           2   
 ENSG00000227232           0           1           1           0           0   
 ENSG00000278267           0           0           0           0           0   
 ENSG00000243485           0           0           0           0           0   
 ENSG00000274890           0           0           0           0           0   
 ...                     ...         ...         ...         ...         ...   
 ENSG00000275028           0           0           0           0           0   
 ENSG00000278806           0           0           0           0           0   
 ENSG00000274152           0           0           0           0           0   
 ENSG00000276666           0           0           0           0           0   
 ENSG00000277917           0           0           0           0           0   
 
                  BCSC_S5_06

In [19]:
data['silver_seq']['counts']

,AD_1_02_1,AD_1_04_1,AD_1_08_1,AD_1_09_1,AD_1_11_1,AD_1_13_1,AD_2_01_1,AD_2_02_1,AD_2_03_1,AD_2_05_1,...,N_7_04_1,N_7_07_1,N_8_01_1,N_8_05_1,N_8_07_1,N_9_00_1,N_9_03_1,N_9_06_1,N_9_08_1,N_9_12_1
ENSG00000223972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000227232,0,0,0,2,0,0,0,0,0,3,...,0,0,3,0,0,0,0,0,0,0
ENSG00000278267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000243485,1,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000274890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000275028,5,3,0,0,0,2,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
ENSG00000278806,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000274152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000276666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
data['toden']['metadata']['Disease'].value_counts()

Disease
AD     170
NCI    164
Name: count, dtype: int64